In [10]:
import numpy as np
import cv2
import dlib
import os
import pandas as pd
import json
from glob import glob
import dlib

In [11]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [12]:
def preprocess_eye_region(frame, eye_coords, target_size=(30, 36)):
    """
    Preprocesses the eye region for the CNN model.
    Args:
        frame: The input image frame (in BGR format).
        eye_coords: Coordinates of the eye region.
        target_size: The target size for each eye region.
    Returns:
        The preprocessed eye region.
    """
    x_min = min(x for x, y in eye_coords)
    x_max = max(x for x, y in eye_coords)
    y_min = min(y for x, y in eye_coords)
    y_max = max(y for x, y in eye_coords)

    # Cropping the eye region based on the extremities of the landmarks
    cropped_eye = frame[y_min:y_max, x_min:x_max]

    # Resizing the cropped eye region to the target size
    resized_eye = cv2.resize(cropped_eye, target_size)

    return resized_eye.astype(np.float32) / 255.0

In [14]:
def get_combined_eyes(frame):
    """
    Detects and combines the eye regions from the frame.
    Args:
        frame: The input image frame.
    Returns:
        The combined eye regions, or None if not detected.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Extract the coordinates for each eye
        left_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
        right_eye = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]

        # Preprocess each eye region
        left_eye_region = preprocess_eye_region(frame, left_eye)

        right_eye_region = preprocess_eye_region(frame, right_eye)

        # Combine the eyes side by side
        combined_eyes = np.hstack([left_eye_region, right_eye_region])

        # Ensure the combined eyes image has the correct shape
        if combined_eyes.shape[1] != 60:
            raise ValueError("Combined eyes region does not match the expected width.")


        return combined_eyes

    return None

In [13]:

def normalize_head_pose(head_pose_data, rotation_scale=180, translation_max_displacement=None):
    """
    Normalizes the head pose data.
    Args:
        head_pose_data: List containing the head pose data (rotation and translation vectors).
        rotation_scale: Maximum value for the rotation vector components (180 for degrees, np.pi for radians).
        translation_max_displacement: A tuple (max_x, max_y, max_z) representing the maximum displacement in each axis. If None, standard deviation normalization will be used.

    Returns:
        Normalized head pose data.
    """
    # Normalize rotation vectors
    normalized_rotation = np.array(head_pose_data[:3]) / rotation_scale

    # Normalize translation vectors
    if translation_max_displacement:
        max_x, max_y, max_z = translation_max_displacement
        normalized_translation = np.array(head_pose_data[3:]) / np.array([max_x, max_y, max_z])
    else:
        # Standard deviation normalization
        translation_vector = np.array(head_pose_data[3:])
        std_dev = np.std(translation_vector)
        mean_val = np.mean(translation_vector)
        normalized_translation = (translation_vector - mean_val) / std_dev

    return np.concatenate([normalized_rotation, normalized_translation]).tolist()

In [6]:
screenWidth, screenHeight =  1707, 960 #pyautogui.size() # Get the size of the primary monitor.

In [15]:

# Assuming normalize_head_pose and get_combined_eyes are defined as before
def get_screen_size(metadata_file_path):
    with open(metadata_file_path, 'r') as f:
        metadata = json.load(f)

        # Check if 'screenData' is a key in the metadata
        if 'screenData' in metadata:
            metadata = metadata['screenData']
        # Otherwise, assume the metadata is already at the top level

        screen_width = metadata.get('screenWidth')
        screen_height = metadata.get('screenHeight')

        if screen_width is None or screen_height is None:
            raise ValueError("Screen size not found in metadata")

        return screen_width, screen_height


def parse_head_pose_data(row):
    # Split the strings and convert to float
    rotation_str, translation_str = row['head_pose'], row['head_translation']
    rotation = [float(x) for x in rotation_str.strip('"').split(',')]
    translation = [float(x) for x in translation_str.strip('"').split(',')]
    return rotation + translation  # Combine into a single list

def prepare_dataset(base_dir):
    X, Y = [], []
    
    column_names = ['image_path', 'cursor_x', 'cursor_y', 'eye_x1', 'eye_y1', 'eye_x2', 'eye_y2', 'eye_x3', 'eye_y3', 'eye_x4', 'eye_y4', 'eye_x5', 'eye_y5', 'eye_x6', 'eye_y6', 'head_pose', 'head_translation']

    for subdir in glob(os.path.join(base_dir, '*/')):
        if 'calibration' in os.path.basename(os.path.normpath(subdir)).lower():
            continue
        print(f"Processing directory: {subdir}")
        metadata_file_path = os.path.join(subdir, 'metadata.json')
        screen_width, screen_height = get_screen_size(metadata_file_path)
        print(f"Screen size: {screen_width}x{screen_height}")

        # Find any CSV file in the directory
        csv_files = glob(os.path.join(subdir, '*.csv'))
        
        csv_files = [f for f in csv_files if 'calibration' not in os.path.basename(f).lower()]

        if not csv_files:
            print(f"No data CSV file found in directory: {subdir}")
            continue
        data_file_path = csv_files[0]

        # Find any directory that contains image files (assuming JPEG for example)
        img_folders = [d for d in os.listdir(subdir) if os.path.isdir(os.path.join(subdir, d)) and glob(os.path.join(subdir, d, '*.png'))]
        if not img_folders:
            print(f"No image folder found that contains images in directory: {subdir}")
            continue
        img_folder = os.path.join(subdir, img_folders[0])

        data = pd.read_csv(data_file_path, header=None, names=column_names)

        for index, row in data.iterrows():
            # Directly use the image path from the dataframe
            img_path = os.path.join(row['image_path'])
            cursor_x, cursor_y = row['cursor_x'], row['cursor_y']
            eye_box_pupil_data = row[3:15].tolist()
            head_pose_data = parse_head_pose_data(row)

            normalized_eye_box_pupil_data = [float(coord) / screen_width if i % 2 == 0 else float(coord) / screen_height for i, coord in enumerate(eye_box_pupil_data)]
            normalized_head_pose_data = normalize_head_pose(head_pose_data)

            # Load the image
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image not found: {img_path}")
                continue

            combined_eyes = get_combined_eyes(img)

            # Append to datasets
            Y.append([cursor_x / screen_width, cursor_y / screen_height] + normalized_eye_box_pupil_data + normalized_head_pose_data)
            X.append(combined_eyes)
    return X, Y

# Example usage:
base_dir = './data'
X, Y = prepare_dataset(base_dir)


Processing directory: ./data\eloise\
Screen size: 1440x900
No data CSV file found in directory: ./data\eloise\
Processing directory: ./data\Hossein\
Screen size: 1536x864
Processing directory: ./data\koala\
Screen size: 1536x864
Processing directory: ./data\lucy\
Screen size: 1440x900
Processing directory: ./data\melissa\
Screen size: 1710x1112
Processing directory: ./data\PerfectUser\
Screen size: 1536x864
Processing directory: ./data\radabeat\
Screen size: 1707x960
Processing directory: ./data\Shaq\
Screen size: 1280x720
Processing directory: ./data\Will\
Screen size: 1707x960
Processing directory: ./data\WilliamOld\
Screen size: 1707x960


In [16]:
#load the model
from keras.models import load_model
model = load_model('./models/eye_gaze_v9_1810_drop_out.2.h5')

In [18]:
Y[-820]

[0.0,
 0.0,
 0.1769185705916813,
 0.26666666666666666,
 0.17164616285881665,
 0.2625,
 0.011716461628588167,
 0.009375,
 0.2038664323374341,
 0.2677083333333333,
 0.19859402460456943,
 0.2635416666666667,
 0.011716461628588167,
 0.009375,
 -0.017729786577407897,
 -0.0003400907956219676,
 0.000905570713799873,
 -0.7901042058971327,
 -0.6207242050384562,
 1.4108284109355884]

In [22]:
#predict one image in training set

canvas = np.zeros((screenHeight, screenWidth, 3), dtype=np.uint8)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)


#load image
img = cv2.imread('./data/WilliamOld/images/eye_head_capture_1703001721.9667974.png')
combined_eyes = get_combined_eyes(img)

#show true gaze from from data_.txt first line
true_gaze = Y[-1][:2]

true_gaze_x_scaled = int(true_gaze[0] * screenWidth)
true_gaze_y_scaled = int(true_gaze[1] * screenHeight)

# Clamping to screen size
true_gaze_x_scaled = max(0, min(true_gaze_x_scaled, screenWidth - 1))
true_gaze_y_scaled = max(0, min(true_gaze_y_scaled, screenHeight - 1))
print(true_gaze_x_scaled, true_gaze_y_scaled)

canvas.fill(0)

# Draw the gaze point on the canvas
cv2.circle(canvas, (true_gaze_x_scaled, true_gaze_y_scaled), 10, (255, 0, 0), -1)

#predict gaze
combined_eyes = np.expand_dims(combined_eyes, axis=0)

predicted_gaze = model.predict(combined_eyes)[0]

gaze_x_scaled = int(predicted_gaze[0] * screenWidth)
gaze_y_scaled = int(predicted_gaze[1] * screenHeight)

# Clamping to screen size
gaze_x_scaled = max(0, min(gaze_x_scaled, screenWidth - 1))
gaze_y_scaled = max(0, min(gaze_y_scaled, screenHeight - 1))


# Draw the gaze point on the canvas in green
cv2.circle(canvas, (gaze_x_scaled, gaze_y_scaled), 10, (0, 255, 0), -1)

# Display the canvas

cv2.imshow('Gaze Tracking on Canvas', canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()


1399 959
1/1 [==============================] - 0s 18ms/step
